### The Stanford Data Project Analysis - Nashville, TN
On a typical day in the United States, police officers make more than 50,000 traffic stops. Our team is gathering, analyzing, and releasing records from millions of traffic stops by law enforcement agencies across the country. Our goal is to help researchers, journalists, and policymakers investigate and improve interactions between police and the public.

### 1. Purpose-of-Analysis
The purpose of the analysis is kind of hollistic approach to explore what we can get out of the data, so I am not sure if there is a specific purpose or question. I will insert all the questions that come to my mind below and will update this notebook frequently. So there are couple of questions we are trying to answer here:
#### 1.1. Is being arrested a gender dependeant?
#### 1.2 Is being arrested a race dependeant? 
#### 1.3 How the race distribution looks like comparing with the actual city race distribution?
#### 1.4. Is the weather a playing factor of increasing/decreasing the tickets rates?
#### 1.5 how the spatial distribution of the tickets looks  like?
#### 1.6 How the spatial distribution of the tickets and gender looks like?
#### 1.7 Do girls make violations at a specific time of the day?
#### 1.8 How the type of violation is distributed around the city?
#### 1.9 Are there any places in the city where speeding is the most common violation?

### 2. Data aquisition
The data of the project is in a form of compressed file hosted online, it will be downloaded and extracted to the project directory. You can always host the data anywhere else and change the pointer in the read csv  line

In [ ]:
import datetime
import pandas as pd
import requests, zipfile, io

In [ ]:
url="https://stacks.stanford.edu/file/druid:hp256wp2687/hp256wp2687_tn_nashville_2019_08_13.csv.zip"

In [ ]:
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
tn_raw=pd.read_csv(z.namelist()[0], low_memory=False)

In [61]:
tn_raw=pd.read_csv('tn_nashville_2019_08_13.csv', parse_dates=[['date', 'time']])

C:\Users\amalattar\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,8,15,16,17,22,23,24,25,29,30,31,32,33,35,36,37,38,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
tn_raw['date_time']

0               2010-10-10 nan
1          2010-10-10 10:00:00
2          2010-10-10 10:00:00
3          2010-10-10 22:00:00
4          2010-10-10 01:00:00
5          2010-10-10 01:00:00
6          2010-10-10 22:02:00
7          2010-10-10 10:04:00
8          2010-10-10 10:05:00
9          2010-10-10 10:05:00
10         2010-10-10 10:07:00
11         2010-10-10 10:10:00
12         2010-10-10 22:10:00
13         2010-10-10 22:11:00
14         2010-10-10 10:13:00
15         2010-10-10 10:13:00
16         2010-10-10 10:13:00
17         2010-10-10 10:15:00
18         2010-10-10 10:16:00
19         2010-10-10 10:16:00
20         2010-10-10 10:17:00
21         2010-10-10 10:17:00
22         2010-10-10 10:18:00
23         2010-10-10 10:18:00
24         2010-10-10 22:18:00
25         2010-10-10 10:19:00
26         2010-10-10 22:19:00
27         2010-10-10 10:20:00
28         2010-10-10 01:02:00
29         2010-10-10 01:02:00
                  ...         
3092321    2016-09-09 09:44:00
3092322 

In [67]:
tn=tn_raw.copy()

### 3. Data Exploration and Cleaning
In this step we will select the column of interest and  will drop all the na values in these columns. We will also drop the columns that we won't need or use.

In [ ]:
tn.head()

In [ ]:
tn.shape

In [ ]:
tn.columns

#### Removing the uneeded columns

In [65]:
tn=tn.iloc[:,[1,2,4,5,9,10,11,13,14,15,16,17,18,19,20,21,22,23,24,]]

In [68]:
tn.columns

Index(['date_time', 'raw_row_number', 'location', 'lat', 'lng', 'precinct',
       'reporting_area', 'zone', 'subject_age', 'subject_race', 'subject_sex',
       'officer_id_hash', 'type', 'violation', 'arrest_made',
       'citation_issued', 'warning_issued', 'outcome', 'contraband_found',
       'contraband_drugs', 'contraband_weapons', 'frisk_performed',
       'search_conducted', 'search_person', 'search_vehicle', 'search_basis',
       'reason_for_stop', 'vehicle_registration_state', 'notes',
       'raw_verbal_warning_issued', 'raw_written_warning_issued',
       'raw_traffic_citation_issued', 'raw_misd_state_citation_issued',
       'raw_suspect_ethnicity', 'raw_driver_searched',
       'raw_passenger_searched', 'raw_search_consent', 'raw_search_arrest',
       'raw_search_warrant', 'raw_search_inventory', 'raw_search_plain_view'],
      dtype='object')

In [ ]:
tn

#### Checking for the nulls in each column

In [ ]:
tn.isnull().sum()

#### Setting Date as index

In [ ]:
tn.set_index('date_time',inplace=True)